<a href="https://colab.research.google.com/github/elisa-negrini/ML-project/blob/main/ML1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision faiss-cpu transformers datasets tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def extract_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        embeddings = model.get_image_features(**inputs)
    return embeddings[0].cpu().numpy()

# Esempio:
embeddings = []
for filename in os.listdir("data"):
    if filename.endswith(".jpg"):
        emb = extract_embedding(os.path.join("data", filename))
        embeddings.append(emb)

np.save("embeddings/features.npy", np.vstack(embeddings))


In [ ]:
!pip install faiss-cpu # faiss-cpu can be used if you need the CPU version
import faiss
import numpy as np
# %%

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.3 MB/s eta 0:00:00


In [ ]:


features = np.load("embeddings/features.npy").astype('float32')
index = faiss.IndexFlatL2(features.shape[1])  # usa cosine se normalizzi
faiss.normalize_L2(features)  # cosine similarity
index.add(features)

D, I = index.search(features, k=5)  # top-5 retrieval
print(I[0])  # indici delle 5 immagini più simili alla prima


[0 1 2 4 3]


In [ ]:
import torch.nn as nn
import torch

class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet50(pretrained=True)
        self.base = nn.Sequential(*list(resnet.children())[:-1])  # rimuovi classifier
        self.fc = nn.Linear(resnet.fc.in_features, 512)

    def forward(self, x):
        x = self.base(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return nn.functional.normalize(x, p=2, dim=1)

# Triplet Loss con hard mining (opzionale)
loss_fn = nn.TripletMarginLoss(margin=0.2, p=2)


In [ ]:
# Esempio base per Recall@K
def recall_at_k(retrieved_ids, true_ids, k):
    correct = sum([true in retrieved[:k] for true, retrieved in zip(true_ids, retrieved_ids)])
    return correct / len(true_ids)
